### Initialization

In [1]:
import os, logging, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage.morphology as skim
%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [2]:
from network_flow_tracker import particle
import trackpy as tp
from network_flow_tracker import linking as NFTLinking
from network_flow_tracker import FlowGraph as FG

In [3]:
from network_flow_tracker import LFBFP
import network_flow_tracker.utils.io as io
import network_flow_tracker.utils.vis as vis

In [4]:
data_group = 'Lightfield'
dataset = 'Zhang2020'
data_root_path = f'C:\\Data\\{data_group}\\{dataset}'
process_data_root = os.path.join(data_root_path, 'processed_data')

info_fp = os.path.join(process_data_root, 'data_info.pickle')
data_info = io.load_data(info_fp)
mm2s_to_pxl2s = 1e3 / data_info['frame_rate_Hz'] / data_info['target_voxel_size_um']
voxel_size_um = data_info['target_voxel_size_um']

In [5]:
debug_Q = False
rm_node_voxel_on_vol_boundary_Q = True
z_idx = 0
# Load the stitched mask 
z_folder_name = data_info['raw_data_folders'][z_idx]
stitch_data_fp = os.path.join(process_data_root, 'itk', 'recon', 'whole_20250614_140908_data.mat')
stitch_data = LFBFP.LFBFProcessing.load_and_parse_annotation_data(stitch_data_fp)
sv_data = LFBFP.LFBFProcessing.get_subvol_data(data_info, stitch_data, z_idx)
sv_disp_vec = sv_data['disp_vec']
mask_size = sv_data['mask_size']
whole_mask_size = stitch_data['mask_size']
vsl_mips = vis.compute_three_view_mip(sv_data['im'])
vsl_mask = sv_data['label_array'] > 0

avg_hematocrit = 0.5
cell_labeled_fraction = 0.1
# visual check 
if debug_Q:
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot()
    # This vascular image has been high-pass-filtered. Large vessel might have black reigon inside. 
    ax.imshow(vsl_mips['yx'], cmap='gray')
    ax.scatter(sv_data['skl_sub'][2], sv_data['skl_sub'][1], 0.25, color='r', alpha=0.3)

Remove 12 node voxels on the sub-volume boundary


In [6]:
skl_r_array = np.zeros(sv_data['mask_size'], np.float16)
skl_r_array.flat[sv_data['skl_ind']] = sv_data['skl_r_v']
skl_label_array = np.zeros(sv_data['mask_size'], dtype=np.int8)
skl_label_array.flat[sv_data['skl_ind']] = sv_data['skl_label_v']
fg = FG.FlowGraph(skl_label_array > 0)
fg.init_nearest_skl_map(vsl_skl_labeled=skl_label_array, vsl_vol_labeled=sv_data['label_array'])
fg

105 skeleton voxels are labeled as vein in vsl_skl_labeled, but as capillary in vsl_vol_labeled. Correct to capillary.


Spatial graph with 753 edges and 402 nodes.

### Load data

In [7]:
# Here we need to match the points to the graph. 
# The sub-graph was cropped according to the regisration vector along the z direction (only!)
# So the points need to be moved along the xy direction: 
data_disp_vec = stitch_data['disp_vec_c'].copy()
data_disp_vec[:, 0] = 0
lfp = LFBFP.LFBFProcessing(data_root_path, data_info)
num_file = data_info['num_files'][z_idx]
# Load data
data_t_list = []
for t_idx in range(num_file):
    tmp_data = io.load_data(lfp.fp_cell_pos(z_idx, t_idx))
    tmp_df, tmp_info = NFTLinking.parse_detection_data(tmp_data, vol_shape=mask_size)
    data_t_list.append(tmp_df)
print("Finish loading cell position data")
data_t_list = NFTLinking.register_detections(data_t_list, disp_vec=data_disp_vec[z_idx], vol_shape=mask_size, inplace_Q=True, verboseQ=False)

Finish loading cell position data


In [8]:
min_peak_snr = 2
mask_dilate_r = 2
max_extra_dist_to_skl = 3
# Detection selection
detections = pd.concat(data_t_list).drop(columns=['sub_0', 'sub_1', 'sub_2', 'eig3', 'bg_std', 'nb_mean'])
detections = detections[detections.peak_nb_snr >= min_peak_snr]
non_bg_mask = skim.binary_dilation(vsl_mask, skim.ball(radius=mask_dilate_r))
in_mask_Q = non_bg_mask.flat[detections.ind.values]
detections = detections[in_mask_Q]

dist_2_skl = fg.nearest_map.ind_to_nearest_dist(detections.ind.values)
nearest_skl_r = skl_r_array.flat[detections.ind.values]
near_skl_Q = (dist_2_skl <= nearest_skl_r + max_extra_dist_to_skl)
detections = detections[near_skl_Q]

detections = detections.reset_index(drop=True)
# Add information
detections['skl_ind'] = fg.nearest_map.ind_to_nearest_ind(detections.ind.values)
detections['edge_label'] = fg.edge.ind_to_label(detections['skl_ind'].values)
detections['node_label'] = fg.node.ind_to_label(detections['skl_ind'].values)
# Define unique detection id 
detections['did'] = np.arange(detections.shape[0], dtype=np.int64)

print(f"Number of detections: {detections.shape[0]}")

Number of detections: 446710


# Tracking

In [9]:
lk_hdl = NFTLinking.Linking(detections, mask_size)

## Stalled particles

In [10]:
search_r = 3
min_vxl_detect = int(1 * data_info['frame_rate_Hz'])
min_select_trace_length = 1
min_hccc_max_length = 10
max_time_gap = 5
min_stationary_trace_length = 0.3 * data_info['frame_rate_Hz']

max_speed_pxl = 2
print(f"Max search speed: {max_speed_pxl / mm2s_to_pxl2s:.2f} mm/s")

stalled_particles = NFTLinking.track_stationary_particles(lk_hdl.data, mask_size=mask_size, min_vxl_detect=min_vxl_detect, 
                                                                    max_speed_pxl=max_speed_pxl, search_r=search_r, 
                                                                    min_trace_length=min_select_trace_length, 
                                                                    min_hccc_max_length=min_hccc_max_length,
                                                                    max_time_gap=max_time_gap, 
                                                                    min_final_trace_length=min_stationary_trace_length)
print(f"Got {len(stalled_particles)} trajectories")
lk_hdl.update_with_particles(stalled_particles, inplace_Q=True)
active_did = lk_hdl.get_active_did(relabel_pid_Q=False, include_particle_endpoints_Q=False)
active_detections = lk_hdl.data.iloc[active_did].sort_values(by='did')

Max search speed: 0.28 mm/s
Finish processing frame 1257. Finish tracking cells. 
Found 3230 trajectories of length at least 1
Found 3230 trajectories of length at least 1
Found 49 high count connected components.
Got 78 trajectories
Number of accepted traces: 78


In [11]:
if debug_Q:
    stalled_p_pos_zyx = np.vstack([p.pos_zyx for p in stalled_particles])
    fig = vis.vis_mips(vsl_mips, stalled_p_pos_zyx.T, fig_title=f"{z_folder_name} # stalled cell: {len(stalled_particles)}")

## Edge-based analysis

In [12]:
# Find the high-count vessel without reliable velocity estimation 
high_freq_ccf = 1 - np.exp(-3/4 * avg_hematocrit * cell_labeled_fraction * 1)
dm_a_para = {'init_bin_size': 2, 'init_dt': 1, 'min_num_div': 3, 'max_num_div': 15, 'max_dt': 16, 
             'min_peak_corr': 0.05, 
             're_est_min_tot_cor_r': 3, 
             'high_freq_ccf': high_freq_ccf}

dm_select_para = {'min_tot_corr_r': 2, 'min_major_corr': 0.25, 'max_cv': 0.8, 'max_ok_std': 2}

para_hfs = {'min_high_v': 10, 'min_hc_num': 1}

dm_a_adj_e_para = {
    'min_peak_corr': dm_a_para['min_peak_corr'], 
    'min_tot_corr_r': dm_select_para['min_tot_corr_r'], 
    'min_major_corr': dm_select_para['min_major_corr'], 
    'min_v2l': 0.75, 
    'vis_Q': False
}
dm_log, dm_avg_v, dm_v_std = NFTLinking.estimate_flow_velocity_using_detection_map(fg, active_detections, para_dm=dm_a_para, 
                                                                                                     para_dms=dm_select_para, para_hfs=para_hfs, 
                                                                                                     para_mea=dm_a_adj_e_para)

Finish processing edge 700. 
Finish analyzing edge detection map.


In [13]:
non_reliable_edge_Q = np.isnan(dm_avg_v)
non_reliable_v_ind = np.concatenate(fg.edge.cc_ind[non_reliable_edge_Q])
non_reliable_v_sub = fg.ind2sub(non_reliable_v_ind)

## Track moving particles 

### Graph-guided tracking without prediction

In [14]:
max_speed_pxl = 15 # ~ 2 mm/s
print(f"Max search speed: {max_speed_pxl / mm2s_to_pxl2s:.2f} mm/s")
fg.configure_trackpy_query(search_range=max_speed_pxl, num_nb=10, predictQ=False)
trace_result = NFTLinking.tracking(active_detections, max_speed_pxl, pos_col=['z', 'y', 'x'], dist_func=fg.trackpy_query, output_type='table')


Max search speed: 2.10 mm/s
Finish configuring trackpy query function.  {'search_range': 15, 'num_nb': 10, 'max_exit_travel': 0, 'predictQ': False, 'gdistQ': True, 'v_error_frac': 0.5, 'v_error_min': 5, 'compare_feature': [], 'feature_cost_max': 10}
Finish processing frame 1257. Finish tracking cells. 


In [15]:
# trace selection 
para_ts = {'mask_size': mask_size, 
           'min_length': 3, 
           'min_cos': 0.25, 
           'min_med_cos': 0.75, 
           'max_ignore_v': 2} 
# tracking-based edge velocity estimation 
para_teve = {'gm_max_num_est': 3,  # maximum number of mixture-of-gaussian fitting (random initialization) 
             'min_data_size': 5,  # minimal data size for fitting
             'min_gmc_dist_std_n': 2} # minimal mixture-of-gaussian normalized mean difference. If 2 components fit the distribution better but their spacing is smaller than this value, re-do the fitting
para_teves = {'min_weight_ratio': 1.5, 
              'min_track_fraction': 0.3, 
              'max_abs_cv': 0.8, 
              'max_std_to_ignore_cv': 1}

edge_tk_v_results, edge_trkb_v, edge_trkb_v_std = NFTLinking.estimate_flow_velocity_using_tracking(fg, trace_result, para_ts, para_teve, para_teves)

Found 39361 trajectories of length at least 3
Finish processing trace 39350. Found 15660 valid trajectories of length at least 3
Finish constructing voxel speed map using (15651 of 15660) traces. 
Finish constructing skeleton speed map.
Finish analyzing flow velocity in edge 710. 
Finish analyzing edge velocity from the tracking result


In [16]:
para_cb_w = {'max_tk_v': 0.75 * max_speed_pxl, 
             'min_std': 2, 
             'max_v_diff_z': 2, 
             'min_nonzero_v': 1, 
             'log_conflict_Q': True}

edge_v_info_0 = NFTLinking.combine_initial_edge_v_estimation(dm_log, dm_avg_v, dm_v_std, edge_trkb_v, edge_trkb_v_std, **para_cb_w)

Edge 151 has v_t greater than the maximum corr-based estimatable velocity. Use tracking-based velocity. 
Edge 328 v_corr -10.83 +/- 2.00, v_t -6.42 +/- 2.33
Edge 432 has v_t greater than the maximum corr-based estimatable velocity. Use tracking-based velocity. 
Edge 656 has v_t greater than the maximum corr-based estimatable velocity. Use tracking-based velocity. 
Edge 671 has v_t greater than the maximum corr-based estimatable velocity. Use tracking-based velocity. 
Number of edges without any good velocity estimation: 128
Number of edges with at least one good velocity estimation: 624


In [17]:
edge_v_info_1 = edge_v_info_0
while True: 
    edge_v_info_1 = fg.infer_edge_velocity_by_node_flow_configuration(edge_v_info_1['v'], edge_v_info_1['std'], verbose_Q=True)
    if len(edge_v_info_1['inferred_edge']) == 0: 
        break

Node 39 has outflow edge [62 81] and no known inflow edge. Infer edge 61 to have velocity 30.303467542325173
Node 60 has outflow edge [ 96 141] and no known inflow edge. Infer edge 103 to have velocity -65.21193196131763
Node 64 has inflow edge [115 135] and no known outflow edge. Infer edge 136 to have velocity 26.684957044330496
Node 123 has inflow edge [250 262] and no known outflow edge. Infer edge 127 to have velocity -11.679630970112004
Node 208 has outflow edge [392 410] and no known inflow edge. Infer edge 405 to have velocity -10.410808827512554
Node 219 has inflow edge [ 39 280] and no known outflow edge. Infer edge 430 to have velocity 15.149214834057302
Node 229 has inflow edge [232 449] and no known outflow edge. Infer edge 412 to have velocity -4.3065725645139965
Node 247 has inflow edge [478 479 487] and no known outflow edge. Infer edge 355 to have velocity -13.516225302427822
Node 264 has inflow edge [492 517] and no known outflow edge. Infer edge 328 to have velocity 

# Prediction based tracking

In [18]:
# trace selection 
para_ts = {'mask_size': mask_size, 
           'min_length': 3, 
           'min_cos': 0.25, 
           'min_med_cos': 0.75, 
           'max_ignore_v': 2} 

para_evu = {'max_v_diff_z': 2, 
            'min_nonzero_v': 1, 
            'min_std': 1, 
            'min_tk_f': 0.1, 
            'min_trust_f': 0.5, 
            'allow_conflicting_Q': True
            }
max_speed_pxl = 20
v_error_frac = 0.75
v_error_min = 10
feature_list = ['peak_int']
result_log = []

fg.configure_trackpy_query(search_range=max_speed_pxl, num_nb=10, max_exit_travel=0, predictQ=True, v_error_frac=0.75, v_error_min=10, compare_feature=feature_list)

Finish configuring trackpy query function.  {'search_range': 20, 'num_nb': 10, 'max_exit_travel': 0, 'predictQ': True, 'gdistQ': True, 'v_error_frac': 0.75, 'v_error_min': 10, 'compare_feature': ['peak_int'], 'feature_cost_max': 10}


In [20]:
for p_est_count in range(1):
    print(f"Start the {p_est_count + 1} round of prediction-based tracking\n")
    if p_est_count == 0: 
        curr_v = edge_v_info_1['v']
        curr_std = edge_v_info_1['std']
        curr_track_frac = None
    else: 
        curr_v_info = result_log[p_est_count - 1]['edge_v_info']
        curr_v = curr_v_info['v']
        curr_std = curr_v_info['std']
        curr_track_frac = curr_v_info['track_frac']

    fg.init_velocity(curr_v, curr_std, curr_track_frac)
    ptrace_result = NFTLinking.tracking(active_detections, max_speed_pxl, pos_col=['z', 'y', 'x'], feature_col=feature_list, dist_func=fg.trackpy_query, output_type='table')
    p_edge_tk_info_0, pedge_trkb_v, pedge_trkb_v_std = NFTLinking.estimate_flow_velocity_using_tracking(fg, ptrace_result, para_ts, para_teve, para_teves)
    
    p_edge_tk_info = p_edge_tk_info_0.copy()
    p_edge_tk_info['est_v'] = pedge_trkb_v
    p_edge_tk_info['est_v_std'] = pedge_trkb_v_std

    updated_e_v_info = NFTLinking.update_edge_flow_estimation(curr_v, curr_std, curr_track_frac, p_edge_tk_info, **para_evu)
    log_result = {'iteration': p_est_count, 'table': ptrace_result, 'edge_tk_info': p_edge_tk_info, 'edge_v_info': updated_e_v_info}
    result_log.append(log_result) 

Start the 1 round of prediction-based tracking

Finish processing frame 1257. Finish tracking cells. 
Found 11145 trajectories of length at least 3
Finish processing trace 11100. Found 5680 valid trajectories of length at least 3
Finish constructing voxel speed map using (5671 of 5680) traces. 
Finish constructing skeleton speed map.
Finish analyzing flow velocity in edge 710. 
Finish analyzing edge velocity from the tracking result
Edge 610: 9.2 +/- 2.6, 19.6 +/- 5.5 does not make sense


In [21]:
time_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
data_dict = {}
data_dict['date'] = time_str
data_dict['graph_version'] = stitch_data_fp
data_dict['data_group'] = data_group
data_dict['dataset'] = dataset
data_dict['data_info'] = data_info
data_dict['tracking_all'] = NFTLinking.merge_detection_table(lk_hdl.data.copy(), ptrace_result)
data_dict['tracking_ns'] = ptrace_result
data_dict['num_pt_iter'] = len(result_log)
data_dict['sv_data'] = sv_data
data_dict['skl_vsl_map'] = fg.vxl_speed_map
data_dict['e_ep_ind'] = np.vstack([ind[[0, -1]] for ind in fg.edge.cc_ind])
data_dict['edge_v_pxl'] = fg.edge_v_pxl
data_dict['edge_v_std_pxl'] = fg.edge_v_std_pxl
data_dict['edge_track_frac'] = fg.edge_track_frac

Number of accepted traces: 29574


In [ ]:
data_folder = os.path.join(process_data_root, 'tracking')
data_fp = os.path.join(data_folder, f"{data_info['raw_data_folders'][z_idx]}_tk_w_pdt_{data_dict['date']}.pickle")
os.makedirs(data_folder, exist_ok=True)
io.save_data(data_fp, data_dict)
print(f"Finish saving {data_fp}")